Exploration of BigBird-CT model generated embeddings that have high cosine similarity to demonstrate the applicability of the methodology

N.B. Strongly recommended to run this Notebook on a GPU

In [3]:
from pathlib import Path
import pandas as pd
from sentence_transformers import SentenceTransformer, util


pd.set_option('max_colwidth', None)

# load full testing data
test_path = Path.cwd().parent.joinpath('data/interim/test_unlabelled.pkl')
test = pd.read_pickle(test_path)

# load our fine-tuned BigBird-CT with in-batch negatives model
model_bigbird_ct_path = Path.cwd().parent.joinpath('models/bigbird-ct')
model = SentenceTransformer(model_bigbird_ct_path)

sentences = test['Concatenated'].tolist()
codes = test['ModuleCode'].tolist()

# get document embeddings for our testing set modules
embeddings = model.encode(sentences,
                          batch_size = 16,
                          show_progress_bar = True)

Batches:   0%|          | 0/54 [00:00<?, ?it/s]

Attention type 'block_sparse' is not possible if sequence_length: 676 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


One application of this project is the discovery of module catalogue entries that are semantically similar to some other catalogue entry of interest. This can be used to many ends, such as course recommendations for students, identification of duplicate teaching, and could even facilitate collaboration between university faculty members including those across different departments.

We will demonstrate an implementation of this here, directly using the cosine similarities of the generated document embeddings. The results of topic modelling could be used to a similar effect, where instead the clusters are used.

We list the ten highest cosine similarity document embeddings to an arbitrarily chosen document embedding, including the self-similarity.

In [23]:
# find the cosine similarity matrix for the embeddings
cos_sim = util.cos_sim(embeddings, embeddings)

# add all pairs to a list, with their cosine similarity score, including self-similarities
all_sentence_combinations = []
for i in range(len(cos_sim) - 1):
    for j in range(i, len(cos_sim)):
        all_sentence_combinations.append([cos_sim[i][j], i, j])

In [39]:
def most_similar_embeddings(document_id):
    '''
    Get the ten highest cosine similarity document embeddings to the document embedding associated with the provided ID
    This includes the self-similarity
    '''
    # get all pairs that feature the document embedding of interest
    similarity_pairs = []
    for score, i, j in all_sentence_combinations:
        if (i == document_id) or (j == document_id):
            similarity_pairs.append([score, i, j])
    # sort the list by descending cosine similarity
    similarity_pairs = sorted(similarity_pairs, key = lambda x: x[0], reverse = True)
    # get ten largest similarity pairs
    similarity_pairs = similarity_pairs[0:10]

    # make dataframe containing details of ten largest similarity pairs
    most_similar_df = pd.DataFrame(columns = ['ModuleCode', 'Document', 'Cosine Similarity'])
    for comparison in similarity_pairs:
        score, i, j = comparison
        if j == document_id:
            to_append = i
        else:
            to_append = j
        most_similar_df.loc[len(most_similar_df)] = [codes[to_append], sentences[to_append], float(score)]

    return most_similar_df

most_similar = most_similar_embeddings(400)

most_similar

ModuleCode  \
0  [CSC8423, CSC8430]   
1           [CSC3632]   
2  [CSC8207, CSC8410]   
3  [CSC8212, CSC8414]   
4           [EEE8119]   
5           [CSC8004]   
6  [LAW3051, LAW3251]   
7           [CSC3131]   
8           [CSC8021]   
9           [CSC8429]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

The first row of the above table includes the document that is being evaluated for similar documents; it is included for comparative purposes.

Here we are analysing the most similar document embeddings to that for modules **CSC8423** and **CSC8430**, which were grouped during preprocessing for essentially represent duplicate modules. This module, generally, discusses security principles and considerations for software engineering, including concepts like data management. Looking in the table, we see **LAW3051** and **LAW3251** have been listed with a cosine similarity of 0.703038, which is fairly high. Indeed, reading the text for this module we see that it refers to the legality of the internet and networked information, referencing concepts such as digital assets.

These modules are semantically similar enough to the point of interest. A student who studied **CSC8423/CSC8430** may wish to learn more on the subject, from more of a legal standpoint; through this methodology they may find **LAW3051/LAW3251** and thus enquire into this other, similar module. Similarly, a module lead for **CSC8423/CSC8430** may wish to encorporate more robust legal content into the module, and might hence consult with a module lead from **LAW3051/LAW3251**.

There are many other applications of the output of this modelling. For example, a lecturer from the school of mathematics, statistics and physics may wish to teach a course on security principles and considerations for mathematical programming, not knowing that a very similar set of teaching exists under **CSC8423/8430** with the school of computing. They could then write a set of prose in the format of the module catalogue entries to detail what their module would contain. This would then get modelled by the Transformer and have its similar document embeddings found. By this, the lecturer would then see that a module similar to the subject of what they wish to teach already exists, **CSC8423/8430**, and may instead discuss with the module lead of this existing course how they might be able to accomodate their wish to teach the subject.